In [24]:
import pandas as pd
import os
from backtesting import Backtest
from strategies.SMACross import SmaCross
from strategies.EmaRsiAtrStrategy import EmaRsiAtrStrategy
from strategies.VWAP_BB_RSI import VWAP_BB_RSI
from strategies.VWAP_BB_RSI_Precomputed import VWAP_BB_RSI_Precomputed, applyIndicators
from strategies.VWAP import VWAP
from strategies.ML import PredictionStrategy

In [18]:
file_name = 'NSE:RELIANCE-EQ 2020-08-01 to 2021-08-03 5m.csv'
interval = '5m'
strategy = PredictionStrategy
start_date = '2021-07-01'
end_date = ''

In [19]:
# this data is used for folder and file name creation on saving the results
strategy_name = strategy.__name__
symbol = file_name.split(' ')[0]

In [20]:

data = pd.read_csv(f'../data/{symbol}/{file_name}',
                   parse_dates=True, 
                   index_col="Date"
                   )

# data = pd.read_csv("/Users/sohail21400/Desktop/trading_machine/data/EURUSD/EURUSD_Candlestick_4_Hour_ASK_05.05.2003-16.10.2021.csv")

data.index = pd.to_datetime(data.index, format="%Y-%m-%d %H:%M:%S")
data.tail()

,Count,Open,High,Low,Close,Volume
Date,,,,,,
2021-08-03 15:05:00,18731,2085.45,2087.00,2084.80,2085.05,148374
2021-08-03 15:10:00,18732,2084.60,2085.75,2084.00,2085.65,169357
2021-08-03 15:15:00,18733,2085.60,2090.90,2085.00,2090.00,254454
2021-08-03 15:20:00,18734,2090.05,2091.00,2088.55,2089.80,463843
2021-08-03 15:25:00,18735,2089.70,2091.00,2088.75,2090.45,219968


In [21]:
if start_date != '':
    data = data.loc[start_date:]
if end_date != '':
    data = data.loc[:end_date]

print(f"Total lenght of data: {len(data)}")
data

Total lenght of data: 1725


,Count,Open,High,Low,Close,Volume
Date,,,,,,
2021-07-01 09:15:00,17011,2118.00,2123.15,2112.00,2114.00,392878
2021-07-01 09:20:00,17012,2114.15,2121.00,2113.10,2119.00,282505
2021-07-01 09:25:00,17013,2119.80,2119.80,2115.00,2115.60,84073
2021-07-01 09:30:00,17014,2115.60,2115.60,2111.10,2114.40,90920
2021-07-01 09:35:00,17015,2114.40,2117.05,2113.05,2113.90,74613
...,...,...,...,...,...,...
2021-08-03 15:05:00,18731,2085.45,2087.00,2084.80,2085.05,148374
2021-08-03 15:10:00,18732,2084.60,2085.75,2084.00,2085.65,169357
2021-08-03 15:15:00,18733,2085.60,2090.90,2085.00,2090.00,254454


In [23]:
cash = 10_000
commission = 0.002
margin = 1.0
trade_on_close = False
hedging = False
exclusive_orders = False

bt = Backtest(
    data,
    strategy,
    cash=cash,
    # commission=commission,
    margin=margin,
    trade_on_close=trade_on_close,
    hedging=hedging,
    exclusive_orders=exclusive_orders,
)

# stats = bt.run(
#   sma_length1=5,
#   sma_length2=10,
#   atr_length=7,
#  )
stats = bt.optimize(
    sma_length1 = range(5, 30, 5),
    sma_length2 = range(10, 45, 5),
    # bband_length = range(10, 20, 2),
    # bband_std = range(1, 3, 1),
    # rsi_length = range(10, 20, 2),
    # vwap_signal_length = range(10, 20, 2),
    atr_length = range(5, 16, 2),
    # maximize='Return [%]'
)

stats

RuntimeError: Indicator "MA40" errored with exception: sma() missing 1 required positional argument: 'close'

In [ ]:
stats["_strategy"]

<Strategy SmaCross(sma_length1=5,sma_length2=10,atr_length=7)>

In [ ]:
# saving the data to a csv file
if not os.path.isdir(f'./results/{strategy_name}'):
    print(f'Created directory for {strategy_name}')
    os.mkdir(f'./results/{strategy_name}')
    
if not os.path.isdir(f'./results/{strategy_name}/{symbol}'):
    print(f'Created directory for {symbol}')
    os.mkdir(f'./results/{strategy_name}/{symbol}')

base_url = f'./results/{strategy_name}/{symbol}/{interval}'
# saving trade records
stats['_trades'].to_csv(f'{base_url}_trades.csv')

# saving equity curve
stats['_equity_curve'].to_csv(f'{base_url}_equity_curve.csv')

In [ ]:

# saving backtesting results
trade_index = stats[stats.index == '_trades'].index
stats = stats.drop(trade_index)
equity_index = stats[stats.index == '_equity_curve'].index
stats = stats.drop(equity_index)

stats.to_csv(f'{base_url}_results.csv')

In [ ]:
bt.plot(filename=f'{base_url}_plot.html')

0:123: execution error: File some object wasn’t found. (-43)


Row(id='1516', ...)